In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
! rm ./test_db3.db

In [3]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v['description'], interval_number=v['interval_number']))
   # components_values = association_proxy(
   #     'components', 'value',
   #     creator=lambda k, v: Component(id=k, description=v['description']))

class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    id = Column(Integer, primary_key=True)
    borehole = Column(String(32), ForeignKey('Boreholes.id'))
    # Note that this could be a numeric ID as well
    interval_number = Column(Integer)
    components = relationship('Component',secondary='Linkintervalcomponent')
    description = Column(String(32))
    
class Component(Base):
    """The Component table"""
    __tablename__ = 'Components'
    id = Column(String(32), primary_key=True)
    intervals = relationship(Interval,secondary='Linkintervalcomponent')
    description = Column(String(32))

class LinkIntervalComponent(Base):
    __tablename__ = 'Linkintervalcomponent'

    int_id = Column(
        Integer, 
        ForeignKey('Intervals.id'), 
        primary_key = True)

    comp_id = Column(
       Integer, 
       ForeignKey('Components.id'), 
       primary_key = True)

In [4]:
engine = create_engine('sqlite:///test_db3.db', echo=True)

In [5]:
Base.metadata.create_all(engine)

2020-12-11 13:57:03,519 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-11 13:57:03,521 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 13:57:03,522 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-11 13:57:03,523 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 13:57:03,525 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-11 13:57:03,525 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 13:57:03,526 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-11 13:57:03,527 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 13:57:03,528 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-11 13:57:03,528 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 13:57:03,529 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Intervals")
2020-12-11 13:57:03,529 INFO sqlalchemy.engine.base.Engine ()
2020-12-11

In [6]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [7]:
from striplog import Lexicon, Striplog

In [8]:
lexicon = Lexicon.default()

In [9]:
borehole_dict = {'F01':'test.las', 'F02':'test.las'} # boreholes to insert into the db

In [10]:
int_id = 0
bh_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(Borehole(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number}})
        interval_number+=1
        int_id += 1
    # print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

In [11]:
components

{0: Component({'lithology': 'siltstone', 'colour': 'grey'}),
 1: Component({'lithology': 'anhydrite'}),
 2: Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}),
 3: Component({'lithology': 'dolomite'}),
 4: Component({'lithology': 'siltstone', 'colour': 'red'}),
 5: Component({'lithology': 'limestone'})}

In [12]:
component_dict

{Component({'lithology': 'siltstone', 'colour': 'grey'}): 0,
 Component({'lithology': 'anhydrite'}): 1,
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}): 2,
 Component({'lithology': 'dolomite'}): 3,
 Component({'lithology': 'siltstone', 'colour': 'red'}): 4,
 Component({'lithology': 'limestone'}): 5}

In [13]:
strip.components

[Component({'lithology': 'siltstone', 'colour': 'grey'}),
 Component({'lithology': 'anhydrite'}),
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}),
 Component({'lithology': 'dolomite'}),
 Component({'lithology': 'siltstone', 'colour': 'red'}),
 Component({'lithology': 'limestone'})]

In [14]:
strip[0].components

[Component({'lithology': 'anhydrite'})]

In [15]:
boreholes

[<__main__.Borehole at 0x7f3462ab35f8>, <__main__.Borehole at 0x7f3462a52eb8>]

In [23]:
for id in components.keys():
    print(id, components[id])

0 {'lithology': 'siltstone', 'colour': 'grey'}
1 {'lithology': 'anhydrite'}
2 {'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}
3 {'lithology': 'dolomite'}
4 {'lithology': 'siltstone', 'colour': 'red'}
5 {'lithology': 'limestone'}


In [46]:
with session_scope() as session:
    for bh in boreholes:
        session.add(bh)
        for iv in bh.intervals:
            print(bh.intervals[iv].components[0])
        
with session_scope() as session:
    for id in components.keys():
        new_compo = Component(id=id, description=components[id].summary()) 
        session.add(new_compo)

2020-12-11 14:14:15,839 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-11 14:14:15,841 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2020-12-11 14:14:15,845 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-11 14:14:15,846 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-11 14:14:15,847 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-11 14:14:15,849 INFO sqlalchemy.engine.base.Engine SELECT "Components".id AS "Components_id", "Components".description AS "Components_description" 
FROM "Components", "Linkintervalcomponent" 
WHERE ? = "Linkintervalcomponent".int_id AND "Components".id = "Linkintervalcomponent".comp_id
2020-12-11 14:14:15,849 INFO sqlalchemy.eng

IndexError: list index out of range